# Import Library and Module necessary for Model

**Tổng quan mô hình**
--
---
- Mô hình Naive Bayes giả định rằng các đặc trưng trong dữ liệu độc lập với nhau.
- Với bài toán phân loại mô hình Naive Bayes đem lại một hiệu suất tốt với cách triển khai đơn giản.

In [118]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

import os

import tensorflow_hub as hub
# tensor flow module
import tensorflow as tf
import tensorflow_probability as tfp

# matplotlib
from matplotlib import colors
from matplotlib import pyplot as plt

# word vectorizor
# first converts the text into a matrix of word counts
# then transforms these counts by normalizing them based on the term frequency
from sklearn.feature_extraction.text import TfidfVectorizer

from google.colab import drive

drive.mount('/content/drive', force_remount=True)
for dirname, _, filenames in os.walk('/content/drive/My Drive/Colab Notebooks/nlp-getting-started'
):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/nlp-getting-started/sample_submission.csv
/content/drive/My Drive/Colab Notebooks/nlp-getting-started/test.csv
/content/drive/My Drive/Colab Notebooks/nlp-getting-started/train.csv


# Data Preprocessing
---
- Trong bài toán này, mô hình Naive Bayes dùng để phân loại xem các tweet có thông báo một thảm hoạ hay không. Ví dụ một tweet đăng bài có từ khoá ```Ablaze``` có 2 nghĩa:
    * Ẩn dụ về một sự kiện, tâm trạng.
    * Thông báo chắc chắn về một thảm hoạ sẽ xảy đến.
- Với cách học tập độc lập từ các đặt trưng, mô hình Naive Bayes là mô hình hoàn hảo cho vấn đề này.
- Quá trình xử lí dữ liệu ở đây qua các bước:
    + Import dữ liệu từ Google Drive.
    + Embedding dữ liệu thành các vector.
    + Chuẩn hoá dữ liệu thành matrics.

## Import Dataset

---
- Bộ dữ liệu phân loại nhị phân được dùng trong mô hình là Natural Language Processing With Disaster Tweets. [Link Drive Dataset ở đây.](https://drive.google.com/drive/folders/19bqWjvVvlYBGucs8zFWv1-xr1FjN_u_7) Hoặc có sẵn file csv trong Folder, thay đổi đường dẫn để lấy Dataset.
- Sau khi Get Data chia Data làm 2 tập train và test.

In [119]:
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/nlp-getting-started/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/nlp-getting-started/test.csv")


## Embeding Tweet Text.

---
- Khi làm việc với dữ liệu là ngôn ngữ tự nhiên, ta phải Embedding các *câu* (sentence), *từ ngữ* (text) thành các *vector*.
- Trong mô hình này ta dùng Tensorflow universal-sentence-encoder để Embedding.

In [121]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/3")
X_train_embeddings = embed(train_df["text"].values)
X_test_embeddings = embed(test_df["text"].values)

## Tensorflow Numpy Matrics
---
- Để có thể huấn luyện mô hình Naive Bayes, ta chuẩn hoá các mẫu dữ liệu thành ma trận với *hàng* (các mẫu độc lập) và *cột* (các đặc trưng) nhằm tối ưu hoá việc tổ chức, xử lý, tính toán. Với bộ dữ liệu lớn hay nhỏ đều có thể dùng được.

In [122]:
X_train_matrix = X_train_embeddings['outputs'].numpy()
X_test_matrix = X_test_embeddings['outputs'].numpy()

Y_train = np.array(tf.constant(train_df["target"]))

# Build And Train Naive Bayes Classifier

## Build Naive Bayes Classifier
---
- # `fit()`: Huấn luyện học máy mô hình Naive Bayes.
- # `predict()`: Phân loại dữ liệu dựa trên mô hình Naive Bayes.

In [123]:
class TFNaiveBayesClassifier:
    dist = None

    # X is the matrix containing the vectors for each sentence
    # y is the list target values in the same order as the X matrix
    def fit(self, X, y):
        unique_y = np.unique(y) # unique target values: 0,1
        print(unique_y)
        # `points_by_class` is a numpy array the size of
        # the number of unique targets.
        # in each item of the list is another list that contains the vector
        # of each sentence from the same target value
        points_by_class = [[np.asarray(X.iloc[x,:]) for x in range(0,len(y)) if y[x] == c] for c in unique_y]

        mean_list=[]
        var_list=[]
        for i in range(0, len(points_by_class)):
            mean_var, var_var = tf.nn.moments(tf.constant(points_by_class[i]), axes=[0])
            mean_list.append(mean_var)
            var_list.append(var_var)
        mean=tf.stack(mean_list, 0)
        var=tf.stack(var_list, 0)
        # Create a 3x2 univariate normal distribution with the
        # known mean and variance
        self.dist = tfp.distributions.Normal(loc=mean, scale=tf.sqrt(var))

    def predict(self, X):
        assert self.dist is not None
        nb_classes, nb_features = map(int, self.dist.scale.shape)

        # uniform priors
        priors = np.log(np.array([1. / nb_classes] * nb_classes), dtype=np.float32)

        # Conditional probabilities log P(x|c)
        # (nb_samples, nb_classes, nb_features)
        all_log_probs = self.dist.log_prob(
            tf.reshape(
                tf.tile(X, [1, nb_classes]), [-1, nb_classes, nb_features]))
        # (nb_samples, nb_classes)
        cond_probs = tf.reduce_sum(all_log_probs, axis=2)

        # posterior log probability, log P(c) + log P(x|c)
        joint_likelihood = tf.add(priors, cond_probs)

        # normalize to get (log)-probabilities
        norm_factor = tf.reduce_logsumexp(
            joint_likelihood, axis=1, keepdims=True)
        log_prob = joint_likelihood - norm_factor
        # exp to get the actual probabilities
        return tf.exp(log_prob)

## Initialize Naive Bayes Model
---
Khởi tạo mô hình.

In [124]:
tf_nb = TFNaiveBayesClassifier()

## Train Naive Bayes With Train Data
---
- Vì thuật toán Naive Bayes huấn luyện với các đặc trưng độc lập nên đầu vào của dữ liệu sẽ được chuyển đổi từ dạng:
    ```
    Data = [
        Sample 1
        Sample 2
        Sample 3
          ...
        Sample 7613
    ]
    ->
    Data = [
        Feature 1
        Feature 2
           ...
        Feature 512
    ]
    ```
- Chuyển đổi tập huấn luyện thành dạng Data Frame.
    ```
    X_train_matrix = [
        [x1 x2 x3 ... x511 x512]
        [y1 y2 y3 ... y511 y512]
                  ...
        [z1 z2 z3 ... z511 z512]
    ]
    ->
    Data Frame = [
        [x1 y1 ... z1]
               ...
        [x512 y512 ... z512]
    ]
    ```
- Bắt đầu huấn luyện với dữ liệu được chuyển đổi.
---

In [106]:
tf_nb.fit(pd.DataFrame(X_train_matrix), Y_train)

[0 1]


# Evaluate Model

## Predict Test Data
---
- Sau khi huấn luyện mô hình tiến hành dự đoán trên tập ```test```
---

In [ ]:
y_pred = tf_nb.predict(X_test_matrix)

In [108]:
predProbGivenText_df = pd.DataFrame(y_pred.numpy())
predProbGivenText_df.head()

,0,1
0,7.993159e-01,0.200697
1,1.958262e-09,1.000000
2,6.392048e-04,0.999390
3,6.165956e-13,1.000000
4,7.065323e-18,1.000000


from matplotlib import pyplot as plt
_df_0[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1[1].plot(kind='hist', bins=20, title=1)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x=0, y=1, s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3[0].plot(kind='line', figsize=(8, 4), title=0)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_4[1].plot(kind='line', figsize=(8, 4), title=1)
plt.gca().spines[['top', 'right']].set_visible(False)

## Lọc ra các ```Unique Keywords``` từ tập ```Train```

In [109]:
uniq_keywords = train_df["keyword"].unique()[1:]
print(len(uniq_keywords))
print(uniq_keywords)

221
['ablaze' 'accident' 'aftershock' 'airplane%20accident' 'ambulance'
 'annihilated' 'annihilation' 'apocalypse' 'armageddon' 'army' 'arson'
 'arsonist' 'attack' 'attacked' 'avalanche' 'battle' 'bioterror'
 'bioterrorism' 'blaze' 'blazing' 'bleeding' 'blew%20up' 'blight'
 'blizzard' 'blood' 'bloody' 'blown%20up' 'body%20bag' 'body%20bagging'
 'body%20bags' 'bomb' 'bombed' 'bombing' 'bridge%20collapse'
 'buildings%20burning' 'buildings%20on%20fire' 'burned' 'burning'
 'burning%20buildings' 'bush%20fires' 'casualties' 'casualty'
 'catastrophe' 'catastrophic' 'chemical%20emergency' 'cliff%20fall'
 'collapse' 'collapsed' 'collide' 'collided' 'collision' 'crash' 'crashed'
 'crush' 'crushed' 'curfew' 'cyclone' 'damage' 'danger' 'dead' 'death'
 'deaths' 'debris' 'deluge' 'deluged' 'demolish' 'demolished' 'demolition'
 'derail' 'derailed' 'derailment' 'desolate' 'desolation' 'destroy'
 'destroyed' 'destruction' 'detonate' 'detonation' 'devastated'
 'devastation' 'disaster' 'displaced' 'droug

## Viết thủ công hàm thay thế các ```Unique Keywords``` bằng từ gần giống với nó, giảm thiểu các unique words.

In [125]:
def replace_keywords(df_og):
    df = df_og.copy()
    df["keyword"] = df["keyword"].replace("ablaze","blaze")
    df["keyword"] = df["keyword"].replace("blazing","blaze")
    df["keyword"] = df["keyword"].replace("annihilated","annihilation")
    df["keyword"] = df["keyword"].replace("attacked","attack")
    df["keyword"] = df["keyword"].replace("bioterror","bioterrorism")
    df["keyword"] = df["keyword"].replace("blown%20up","blew%20up")
    df["keyword"] = df["keyword"].replace("bloody","blood")
    df["keyword"] = df["keyword"].replace("bleeding","blood")
    df["keyword"] = df["keyword"].replace("body%20bags","body%20bag")
    df["keyword"] = df["keyword"].replace("body%20bagging","body%20bag")
    df["keyword"] = df["keyword"].replace("bombed","bomb")
    df["keyword"] = df["keyword"].replace("bombing","bomb")
    df["keyword"] = df["keyword"].replace("burning%20buildings","buildings%20burning")
    df["keyword"] = df["keyword"].replace("buildings%20on%20fire","buildings%20burning")
    df["keyword"] = df["keyword"].replace("burned","burning")
    df["keyword"] = df["keyword"].replace("casualties","casualty")
    df["keyword"] = df["keyword"].replace("catastrophe","catastrophic")
    df["keyword"] = df["keyword"].replace("collapse","collapsed")
    df["keyword"] = df["keyword"].replace("collide","collision")
    df["keyword"] = df["keyword"].replace("collided","collision")
    df["keyword"] = df["keyword"].replace("crash","crashed")
    df["keyword"] = df["keyword"].replace("crush","crushed")
    df["keyword"] = df["keyword"].replace("dead","death")
    df["keyword"] = df["keyword"].replace("deaths","death")
    df["keyword"] = df["keyword"].replace("deluge","deluged")
    df["keyword"] = df["keyword"].replace("demolished","demolish")
    df["keyword"] = df["keyword"].replace("demolition","demolish")
    df["keyword"] = df["keyword"].replace("derailment","derail")
    df["keyword"] = df["keyword"].replace("derailed","derail")
    df["keyword"] = df["keyword"].replace("desolation","desolate")
    df["keyword"] = df["keyword"].replace("destroyed","destroy")
    df["keyword"] = df["keyword"].replace("destruction","destroy")
    df["keyword"] = df["keyword"].replace("detonate","detonation")
    df["keyword"] = df["keyword"].replace("devastated","devastation")
    df["keyword"] = df["keyword"].replace("drowned","drown")
    df["keyword"] = df["keyword"].replace("drowning","drown")
    df["keyword"] = df["keyword"].replace("electrocute","electrocuted")
    df["keyword"] = df["keyword"].replace("evacuated","evacuate")
    df["keyword"] = df["keyword"].replace("evacuation","evacuate")
    df["keyword"] = df["keyword"].replace("explode","explosion")
    df["keyword"] = df["keyword"].replace("exploded","explosion")
    df["keyword"] = df["keyword"].replace("fatality","fatalities")
    df["keyword"] = df["keyword"].replace("floods","flood")
    df["keyword"] = df["keyword"].replace("flooding","flood")
    df["keyword"] = df["keyword"].replace("bush%20fires","forest%20fire")
    df["keyword"] = df["keyword"].replace("forest%20fires","forest%20fire")
    df["keyword"] = df["keyword"].replace("hailstorm","hail")
    df["keyword"] = df["keyword"].replace("hazardous","hazard")
    df["keyword"] = df["keyword"].replace("hijacking","hijack")
    df["keyword"] = df["keyword"].replace("hijacker","hijack")
    df["keyword"] = df["keyword"].replace("hostages","hostage")
    df["keyword"] = df["keyword"].replace("injured","injury")
    df["keyword"] = df["keyword"].replace("injures","injury")
    df["keyword"] = df["keyword"].replace("inundated","inundation")
    df["keyword"] = df["keyword"].replace("mass%20murderer","mass%20murder")
    df["keyword"] = df["keyword"].replace("obliterated","obliterate")
    df["keyword"] = df["keyword"].replace("obliteration","obliterate")
    df["keyword"] = df["keyword"].replace("panicking","panic")
    df["keyword"] = df["keyword"].replace("quarantined","quarantine")
    df["keyword"] = df["keyword"].replace("rescuers","rescue")
    df["keyword"] = df["keyword"].replace("rescued","rescue")
    df["keyword"] = df["keyword"].replace("rioting","riot")
    df["keyword"] = df["keyword"].replace("dust%20storm","sandstorm")
    df["keyword"] = df["keyword"].replace("screamed","screams")
    df["keyword"] = df["keyword"].replace("screaming","screams")
    df["keyword"] = df["keyword"].replace("sirens","siren")
    df["keyword"] = df["keyword"].replace("suicide%20bomb","suicide%20bomber")
    df["keyword"] = df["keyword"].replace("suicide%20bombing","suicide%20bomber")
    df["keyword"] = df["keyword"].replace("survived","survive")
    df["keyword"] = df["keyword"].replace("survivors","survive")
    df["keyword"] = df["keyword"].replace("terrorism","terrorist")
    df["keyword"] = df["keyword"].replace("thunderstorm","thunder")
    df["keyword"] = df["keyword"].replace("traumatised","trauma")
    df["keyword"] = df["keyword"].replace("twister","tornado")
    df["keyword"] = df["keyword"].replace("typhoon","hurricane")
    df["keyword"] = df["keyword"].replace("weapons","weapon")
    df["keyword"] = df["keyword"].replace("wild%20fires","wildfire")
    df["keyword"] = df["keyword"].replace("wounded","wounds")
    df["keyword"] = df["keyword"].replace("wrecked","wreckage")
    df["keyword"] = df["keyword"].replace("wreck","wreckage")
    return(df)

## Thay thế các unique keywords trong tập ```Train``` và ```Test``` bằng hàm vừa viết.

In [126]:
train_df = replace_keywords(train_df)
test_df = replace_keywords(test_df)

## Tính xác xuất cho từng ```Sample``` với ```Keywords``` cụ thể.

In [127]:
uniq_keywords = train_df["keyword"].unique()[1:]
kword_resArr = []
print(len(uniq_keywords))
for kword in uniq_keywords:
    kword_df = train_df.loc[train_df["keyword"] == kword,: ]
    total_kword = float(len(kword_df))
    target0_n = float(len(kword_df.loc[kword_df["target"]==0,:]))
    target1_n = float(len(kword_df.loc[kword_df["target"]==1,:]))
    kword_prob_df = pd.DataFrame({'keyword':[kword],
                                 "keywordPred0": [target0_n/total_kword],
                                 "keywordPred1": [target1_n/total_kword]})
    kword_resArr.append(kword_prob_df)
predProbGivenKeyWord_df= pd.concat(kword_resArr)
predProbGivenKeyWord_df.head()

142


,keyword,keywordPred0,keywordPred1
0,blaze,0.824074,0.175926
0,accident,0.314286,0.685714
0,aftershock,1.000000,0.000000
0,airplane%20accident,0.142857,0.857143
0,ambulance,0.473684,0.526316


# Tạo File ```Submission```.

## Tính toán khả năng của Keywords dựa trên tập ```Test```.

In [128]:
test_df["textprob0"]=predProbGivenText_df.loc[:,0].copy()
test_df["textprob1"]=predProbGivenText_df.loc[:,1].copy()
test_df.head()

,id,keyword,location,text,textprob0,textprob1
0,0,NaN,NaN,Just happened a terrible car crash,7.993159e-01,0.200697
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1.958262e-09,1.000000
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",6.392048e-04,0.999390
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,6.165956e-13,1.000000
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,7.065323e-18,1.000000


## Lấy xác xuất với Keywords. Nếu câu không có Keywords thì khả năng là 50%.

In [129]:
test_df = test_df.merge(predProbGivenKeyWord_df, how='left', on="keyword")
test_df["keywordPred0"]=test_df["keywordPred0"].fillna(0.5)
test_df["keywordPred1"]=test_df["keywordPred1"].fillna(0.5)

## Tính toán khả năng dựa trên ```Text``` và ```Keywords```. Chọn phương pháp dựa trên dự đoán có dự đoán tốt hơn. Sau đó tạo Submission File dựa trên phương pháp dự đoán đã chọn.

In [132]:
test_df["pred0"]=test_df["textprob0"]*test_df["keywordPred0"]
test_df["pred1"]=test_df["textprob1"]*test_df["keywordPred1"]
test_df["target"]=test_df["pred1"]>test_df["pred0"]
test_df["target"] = test_df["target"].astype(np.int64)
submission_df = test_df.loc[:,["id","target"]]
submission_df.head()
submission_df.to_csv("submission.csv",index=False)